## Threading enumerate

it is not necessary to retain an explicit handle to all the daemon threads in order to ensure they have completed before exiting the main process.

`threading.enumerate()` returns a list of all Thread objects currently alive. The list includes daemonic threads, dummy thread objects created by current_thread(), and the main thread. it excludes terminated threads and threads that have not yet been started.

In [1]:
import threading
import time
import logging
import random   # to wait for ranodom time for each thread.

Now we need to cofig our logging format and level.

In [2]:
logging.basicConfig(level=logging.DEBUG,
                   format='%(threadName)s: %(message)s')

# a function to for logging 
def log(msg):
    logging.debug(msg)

Now we will create a function to which will be called by each thread.

In [3]:
def func():
    
    r = random.randint(30,60)
    log(f'sleeping in {r} seconds!')
    time.sleep(r)
    log('ending')
    return

Now we will make threads and run them in the main section.

since in jupyter notebook we have some extra threads running in the background so we need to avoide those threads for that we will store them in a list to ignore them.

In [4]:
pre_threads = [t for t in threading.enumerate()]
pre_threads

[<_MainThread(MainThread, started 22048)>,
 <Thread(IOPub, started daemon 11324)>,
 <Heartbeat(Heartbeat, started daemon 13952)>,
 <ControlThread(Control, started daemon 25924)>,
 <HistorySavingThread(IPythonHistorySavingThread, started 19836)>,
 <ParentPollerWindows(Thread-4, started daemon 13784)>]

Now we have all the thread which are not needed.

In [11]:
if __name__ == "__main__":
    
    # first we will create some threads.
    # 3 threads as daemong and 3 threads as non-daemon
    for i in range(3):
        t = threading.Thread(target=func, name=f"Dameon-{i}",
                            daemon=True)
        t.start()
     
    # 3 non-daemon thread
    for i in range(3):
        t = threading.Thread(target=func, name=f"NonDameon-{i}")
        t.start()
        
    # Now we will get the Main thread which can be get by using the 
    # threading.current_thread() method.
    main_thread = threading.current_thread()
    
    # Now we will join each thread non-daemon and daemon thread to main 
    # thread.
    for i in threading.enumerate():
        print(i)
    
    for thread in threading.enumerate():
        
        if thread in pre_threads:
            continue
        
        log(f'joining thread {thread.getName()}')
        thread.join()
    
    for i in threading.enumerate():
        print(i)

Dameon-0: sleeping in 33 seconds!
Dameon-1: sleeping in 56 seconds!
Dameon-2: sleeping in 52 seconds!
NonDameon-0: sleeping in 41 seconds!
NonDameon-1: sleeping in 36 seconds!
NonDameon-2: sleeping in 39 seconds!
MainThread: joining thread Dameon-0


<_MainThread(MainThread, started 22048)>
<Thread(IOPub, started daemon 11324)>
<Heartbeat(Heartbeat, started daemon 13952)>
<ControlThread(Control, started daemon 25924)>
<HistorySavingThread(IPythonHistorySavingThread, started 19836)>
<ParentPollerWindows(Thread-4, started daemon 13784)>
<Thread(Dameon-0, started daemon 9560)>
<Thread(Dameon-1, started daemon 23248)>
<Thread(Dameon-2, started daemon 12060)>
<Thread(NonDameon-0, started 19912)>
<Thread(NonDameon-1, started 10760)>
<Thread(NonDameon-2, started 4688)>


Dameon-0: ending
MainThread: joining thread Dameon-1
NonDameon-1: ending
NonDameon-2: ending
NonDameon-0: ending
Dameon-2: ending
Dameon-1: ending
MainThread: joining thread Dameon-2
MainThread: joining thread NonDameon-0
MainThread: joining thread NonDameon-1
MainThread: joining thread NonDameon-2


<_MainThread(MainThread, started 22048)>
<Thread(IOPub, started daemon 11324)>
<Heartbeat(Heartbeat, started daemon 13952)>
<ControlThread(Control, started daemon 25924)>
<HistorySavingThread(IPythonHistorySavingThread, started 19836)>
<ParentPollerWindows(Thread-4, started daemon 13784)>


As you can see that we have 6 threads 3-Daemon and 3-nonDaemon and we access them by using the threading.enumerate() method, which is very usefull to access any thread which is linked to the current program or MainThread.

At the end of the all thread processing we can see the list of all active thread in that we don't see our 6 threads created by us.

The reason of not being included in the list is that they are not alive threads and the enumerate() method only gives the alive threads.